In [28]:
! nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 3090 (UUID: GPU-72fdbd81-da45-b750-3719-ae5877e26726)


In [29]:
import multiprocessing as mp
num_cpu = mp.cpu_count()
num_cpu

12

In [30]:
_exp_name = "resnet"

In [31]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import random
import wandb

In [32]:
! ls

CNN1.5_best.ckpt	CNN4_fold_1_log.txt	prediction_CNN1_dropout.csv
CNN1.5.ipynb		CNN4_fold_2_log.txt	prediction_CNN1_noBN.csv
CNN1.5_log.txt		CNN4_fold_3_log.txt	prediction_CNN2.csv
CNN1_dropout_best.ckpt	CNN4_fold_4_log.txt	prediction_CNN3.csv
CNN1_dropout.ipynb	cnn-resnet_focal.ipynb	prediction_CNN4.csv
CNN1_dropout_log.txt	cnn-resnet.ipynb	prediction_resnet.csv
CNN1.ipynb		Fold_1_best.ckpt	resnet1.ipynb
CNN1_log.txt		Fold_2_best.ckpt	resnet2.ipynb
CNN1_noBN_best.ckpt	Fold_3_best.ckpt	resnet_fold_1_log.txt
CNN1_noBN.ipynb		Fold_4_best.ckpt	resnet_fold_2_log.txt
CNN2_best.ckpt		food-11			resnet_fold_3_log.txt
CNN2.ipynb		food-11.zip		resnet_fold_4_log.txt
CNN2_log.txt		key.txt			resnet.ipynb
CNN3_fold_1_log.txt	__MACOSX		true_labels.csv
CNN3_fold_2_log.txt	MLP			wandb
CNN3_fold_3_log.txt	prediction_CNN1.5.csv	xgb_gridsearch.ipynb
CNN3_fold_4_log.txt	prediction_CNN1.csv	xgb_gridsearch.py


In [33]:
myseed = 4012  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

## **Transforms**
Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Please refer to PyTorch official website for details about different transforms.

In [34]:
test_tfm = transforms.Compose([
    # (height = width = 128)
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

train_tfm = transforms.Compose([
    # (height = width = 128)
    #transforms.CenterCrop()
    transforms.RandomResizedCrop((128, 128), scale=(0.7, 1.0)),
    #transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
    #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.RandomRotation(180),
    transforms.RandomAffine(30),
    #transforms.RandomInvert(p=0.2),
    #transforms.RandomPosterize(bits=2),
    #transforms.RandomSolarize(threshold=192.0, p=0.2),
    #transforms.RandomEqualize(p=0.2),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    #transforms.RandomApply(torch.nn.ModuleList([]))
])


## **Datasets**
The data is labelled by the name, so we load images and label while calling '__getitem__'

In [35]:
class FoodDataset(Dataset):

    def __init__(self,path=None,tfm=test_tfm,files=None):
        super(FoodDataset).__init__()
        self.path = path
        if path:
            self.files = sorted([os.path.join(path, x) for x in os.listdir(path) if x.endswith(".jpg")])
        else:
            self.files = files
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label

In [36]:
class Residual_Block(nn.Module):
    def __init__(self, ic, oc, stride=1):
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(ic, oc, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(oc),
            nn.ReLU(inplace=True)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(oc, oc, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(oc),
        )
        
        self.relu = nn.ReLU(inplace=True)
    
        self.downsample = None
        if stride != 1 or (ic != oc):
            self.downsample = nn.Sequential(
                nn.Conv2d(ic, oc, kernel_size=1, stride=stride),
                nn.BatchNorm2d(oc),
            )
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        
        if self.downsample:
            residual = self.downsample(x)
            
        out += residual
        return self.relu(out)

class Classifier(nn.Module):
    def __init__(self, block, num_layers, num_classes=11):
        super().__init__()
        self.preconv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
        )
        
        self.layer0 = self.make_residual(block, 32, 64,  num_layers[0], stride=2)
        self.layer1 = self.make_residual(block, 64, 128, num_layers[1], stride=2)
        self.layer2 = self.make_residual(block, 128, 256, num_layers[2], stride=2)
        self.layer3 = self.make_residual(block, 256, 512, num_layers[3], stride=2)
        
#         self.avgpool = nn.AvgPool2d(2)
        
        self.fc = nn.Sequential(            
            nn.Dropout(0.4),
            nn.Linear(512*4*4, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(512, 11),
        )
        
        
    def make_residual(self, block, ic, oc, num_layer, stride=1):
        layers = []
        layers.append(block(ic, oc, stride))
        for i in range(1, num_layer):
            layers.append(block(oc, oc))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        # [3, 128, 128]
        out = self.preconv(x)  # [32, 64, 64]
        out = self.layer0(out) # [64, 32, 32]
        out = self.layer1(out) # [128, 16, 16]
        out = self.layer2(out) # [256, 8, 8]
        out = self.layer3(out) # [512, 4, 4]
#         out = self.avgpool(out) # [512, 2, 2]
        out = self.fc(out.view(out.size(0), -1)) 
        return out

In [37]:
batch_size = 128
num_layers = [2, 4, 4, 1] # residual number layers

n_epochs = 300
patience = 20 # If no improvement in 'patience' epochs, early stop

k_fold = 4

In [38]:
train_dir = "./food-11/training"
val_dir = "./food-11/validation"

train_files = [os.path.join(train_dir, x) for x in os.listdir(train_dir) if x.endswith('.jpg')]
val_files = [os.path.join(val_dir, x) for x in os.listdir(val_dir) if x.endswith('.jpg')]
total_files = train_files + val_files
random.seed(myseed)
random.shuffle(total_files)

num = len(total_files) // k_fold
len(total_files)

13296

In [39]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

test_fold = k_fold

for i in range(test_fold):
    fold = i+1
    print(f'\n\nStarting Fold: {fold} ********************************************')
    model = Classifier(Residual_Block, num_layers).to(device)
    print(next(model.parameters()).device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0004, weight_decay=1e-5) 
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=1)
    stale = 0
    best_acc = 0
    
    val_data = total_files[i*num: (i+1)*num]
    train_data = total_files[:i*num] + total_files[(i+1)*num:]
    
    train_set = FoodDataset(tfm=train_tfm, files=train_data)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_cpu, pin_memory=True)
    
    valid_set = FoodDataset(tfm=test_tfm, files=val_data)
    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=num_cpu, pin_memory=True)
    
    wandb.init(
        project="Food-11",
        config={
            "learning rate": 0.0004,
            "architecture": "CNN",
            "epochs": n_epochs,
            "batch_size": batch_size,
            "image_dim": 128,
            "T_0": 20,
            "T_mult": 1,
        }
    )

    for epoch in range(n_epochs):
    
        # ---------- Training ----------
        # Make sure the model is in train mode before training.
        model.train()
    
        # These are used to record information in training.
        train_loss = []
        train_accs = []
        lr = optimizer.param_groups[0]["lr"]
        
        pbar = tqdm(train_loader)
        pbar.set_description(f'T: {epoch+1:03d}/{n_epochs:03d}')
        for batch in pbar:
    
            # A batch consists of image data and corresponding labels.
            imgs, labels = batch
            #imgs = imgs.half()
            #print(imgs.shape,labels.shape)
    
            # Forward the data. (Make sure data and model are on the same device.)
            logits = model(imgs.to(device))
    
            # Calculate the cross-entropy loss.
            # We don't need to apply softmax before computing cross-entropy as it is done automatically.
            loss = criterion(logits, labels.to(device))
    
            # Gradients stored in the parameters in the previous step should be cleared out first.
            optimizer.zero_grad()
    
            # Compute the gradients for parameters.
            loss.backward()
    
            # Clip the gradient norms for stable training.
            grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
    
            # Update the parameters with computed gradients.
            optimizer.step()
    
            # Compute the accuracy for current batch.
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
    
            # Record the loss and accuracy.
            wandb.log({"step_training_loss": loss.item()})
            wandb.log({"step_training_accuracy": acc})
            train_loss.append(loss.item())
            train_accs.append(acc)
            pbar.set_postfix({'lr':lr, 'b_loss':loss.item(), 'b_acc':acc.item(),
                    'loss':sum(train_loss)/len(train_loss), 'acc': sum(train_accs).item()/len(train_accs)})
            
        train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_accs) / len(train_accs)
        # Print the information.
        print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

        wandb.log({"learning rate": scheduler.get_last_lr()[0]})

        scheduler.step()
        
        
        # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
        model.eval()
    
        # These are used to record information in validation.
        valid_loss = []
        valid_accs = []
    
        # Iterate the validation set by batches.
        pbar = tqdm(valid_loader)
        pbar.set_description(f'V: {epoch+1:03d}/{n_epochs:03d}')
        for batch in pbar:

            # A batch consists of image data and corresponding labels.
            imgs, labels = batch
            #imgs = imgs.half()
    
            # We don't need gradient in validation.
            # Using torch.no_grad() accelerates the forward process.
            with torch.no_grad():
                logits = model(imgs.to(device))
    
            # We can still compute the loss (but not the gradient).
            loss = criterion(logits, labels.to(device))
    
            # Compute the accuracy for current batch.
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
    
            # Record the loss and accuracy.
            wandb.log({"step_validation_loss": loss.item()})
            wandb.log({"step_validation_accuracy": acc})
            valid_loss.append(loss.item())
            valid_accs.append(acc)
            pbar.set_postfix({'v_loss':sum(valid_loss)/len(valid_loss), 
                              'v_acc': sum(valid_accs).item()/len(valid_accs)})
        
            #break
    
        # The average loss and accuracy for entire validation set is the average of the recorded values.
        valid_loss = sum(valid_loss) / len(valid_loss)
        valid_acc = sum(valid_accs) / len(valid_accs)

        # Print the information.
        print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

        # update logs
        if valid_acc > best_acc:
            with open(f"{_exp_name}_fold_{fold}_log.txt","a") as f:
                newline = '\n'
                item = f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best{newline}"
                f.write(item)
                print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
        else:
            with open(f"{_exp_name}_fold_{fold}_log.txt","a") as f:
                newline = '\n'
                item = f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}{newline}"
                f.write(item)
                print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
        
        wandb.log({
            "average_training_loss": train_loss,
            "average_validation_loss": valid_loss,
        })

        # save models
        if valid_acc > best_acc:
            print(f"Best model found at fold {fold} epoch {epoch+1}, acc={valid_acc:.5f}, saving model")
            torch.save(model.state_dict(), f"Fold_{fold}_best.ckpt")
            # only save best to prevent output memory exceed error
            best_acc = valid_acc
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f"No improvment {patience} consecutive epochs, early stopping")
                break
    wandb.finish()

cuda


Starting Fold: 1 ********************************************
cuda:0


T: 001/300: 100%|█| 78/78 [00:08<00:00,  8.89it/s, lr=0.0004, b_loss=2, b_acc=0.


[ Train | 001/300 ] loss = 2.14444, acc = 0.24079


V: 001/300: 100%|██████| 26/26 [00:02<00:00, 11.76it/s, v_loss=2.21, v_acc=0.27]


[ Valid | 001/300 ] loss = 2.20678, acc = 0.26954
[ Valid | 001/300 ] loss = 2.20678, acc = 0.26954 -> best
Best model found at fold 1 epoch 1, acc=0.26954, saving model


...

T: 159/300: 100%|█| 78/78 [00:08<00:00,  9.28it/s, lr=9.79e-6, b_loss=0.16, b_ac


[ Train | 159/300 ] loss = 0.29101, acc = 0.90028


V: 159/300: 100%|████| 26/26 [00:02<00:00, 11.86it/s, v_loss=0.797, v_acc=0.778]

[ Valid | 159/300 ] loss = 0.79697, acc = 0.77767
[ Valid | 159/300 ] loss = 0.79697, acc = 0.77767
No improvment 20 consecutive epochs, early stopping


average_training_loss,█▇▆▅▄▅▅▄▄▃▄▄▃▃▃▄▃▃▂▂▃▃▂▂▂▃▃▂▂▂▃▂▂▂▁▂▂▂▁▁
average_validation_loss,█▅▅▄▃▄▃▄▂▂▃▄▂▂▁▃▂▂▁▁▃▂▂▂▁▃▂▁▂▁▂▂▂▁▁▃▄▂▁▁
learning rate,█▇▅▃▁█▇▅▃▁█▇▅▃▁█▇▅▃▁█▇▅▃▁█▇▅▃▁█▇▅▃▁█▇▅▃▁
step_training_accuracy,▁▂▃▄▄▃▃▅▅▆▄▅▅▅▇▅▅▅▆▇▅▆▆▇▆▇▆▇▇▇▆▆▇▇█▇▇▇▇█
step_training_loss,█▇▆▅▄▆▆▄▄▄▅▄▄▃▃▃▃▄▃▂▄▃▂▂▂▃▃▂▁▁▃▂▂▁▁▂▁▁▁▁
step_validation_accuracy,▁▁▂▄▅▄▃▅▇▆▆▃▅▇▇▄▆▆▆▇▄▄▅█▇▅▆▇▇█▆▇▇▇█▆▅▇▇▇
step_validation_loss,▇█▆▅▄▅▇▅▂▃▃▅▃▂▂▄▃▃▃▂▅▇▄▂▂▅▃▄▂▁▃▂▂▂▁▄▅▄▃▃
average_training_loss,0.29101
average_validation_loss,0.79697
learning rate,1e-05
step_training_accuracy,0.94828


In [40]:
test_dir = "./food-11/test"
test_set = FoodDataset(test_dir, tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_cpu, pin_memory=True)

# Testing and generate prediction CSV

In [41]:
device = "cuda" if torch.cuda.is_available() else "cpu"

test_fold = k_fold

models = []
for i in range(test_fold):
    fold = i + 1
    model_best = Classifier(Residual_Block, num_layers).to(device)
    model_best.load_state_dict(torch.load(f"Fold_{fold}_best.ckpt"))
    model_best.eval()
    models.append(model_best)

prediction = []
test_accs = []
true_labels = []
with torch.no_grad():
    for data, labels in test_loader:
        test_preds = [] 
        for model_best in models:
            test_preds.append(model_best(data.to(device)).cpu().data.numpy())
        
        preds = []
        for i in range(test_fold):
            temp_pred = np.argmax(test_preds[i], axis=1).tolist()
            preds.append(temp_pred)
        pred = list(zip(*preds))
        pred = np.array(pred)
        choice = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=pred)

        test_preds = torch.from_numpy(choice).to(device)
        acc = (test_preds == labels.to(device)).float().mean()
        test_accs.append(acc)
        prediction += choice.squeeze().tolist()
        true_labels.extend(labels.tolist())

test_acc = sum(test_accs) / len(test_accs)
print(f"Test accurary: {test_acc}")

Test accurary: 0.8427448868751526


In [42]:
#create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = prediction
df.to_csv("prediction_resnet.csv",index = False)

df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = true_labels
df.to_csv("true_labels.csv",index = False)